# Тема 3. Выбрать марку автомобиля, скачать не менее 500 объявлений о продаже и построить модель ценообразования.
• Найти и прочитать данные для исследования. \
• Построить вероятностную модель. \
• Проконсультироваться с лектором о выборе модели. \
• Исследовать значимость параметров модели и исключить незначимые. \
• Исследовать адекватность модели статистическим тестом 
и/или стохастическим моделированием, кросс-валидацией. \
• На основании этого исследования исправить модель, а затем 
проверить адекватность исправленной модели.


В качестве марки автомобиля выберем Toyota.

Типы коробок передач:
- Механическая коробка передач (Manual Transmission, MT)
- Роботизированная коробка передач (Automated Manual Transmission, AMT)
- Автоматическая коробка передач (Automatic Transmission, AT)
- Вариатор (Continuously Variable Transmission, CVT)

In [42]:
import re
from collections import Counter
from time import sleep

import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [44]:
AUTO_RU_BASE_URL = "https://auto.ru/cars/{brand}/all/"
BRAND = "mazda"

BUTTON_CLASS = "ListingPagination__moreButton"
N_PAGES = 40

In [45]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
ac = ActionChains(driver)

url = AUTO_RU_BASE_URL.format(brand=BRAND)
driver.get(url)
wait.until(
    EC.presence_of_element_located((By.CLASS_NAME, "ListingCars_outputType_list"))
)
button = driver.find_element(By.CLASS_NAME, BUTTON_CLASS)

for n_page in range(1, N_PAGES):
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, BUTTON_CLASS)))
    ac.move_to_element(button).click().perform()
    sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")

driver.quit()

In [62]:
class Car:
    def __init__(
        self,
        name: str,
        price: str,
        km_age: str,
        year: str,
        engine_characteristics: str,
        gearbox: str,
        car_body: str,
        transmission_type: str,
        color: str,
        link: str,
    ) -> None:
        self.name = name
        self.price = self.digit_serializer(price)
        self.km_age = self.digit_serializer(km_age)
        self.year = int(year)

        self.engine_volume = float(engine_characteristics.split("/")[0].strip().split()[0])
        self.engine_power = self.digit_serializer(engine_characteristics.split("/")[1].strip())
        self.fuel_type = engine_characteristics.split("/")[2].strip().lower()

        self.gearbox = gearbox
        self.car_body = car_body
        self.transmission_type = transmission_type
        self.color = color
        self.link = link

    def __repr__(self) -> str:
        _str = f"{self.name}: "
        _str += f"{self.price}₽, {self.km_age} км, {self.year}, "
        _str += f"{self.engine_volume}, {self.engine_power}, "
        _str += f"{self.fuel_type}, {self.gearbox}, {self.car_body}, "
        _str += f"{self.transmission_type}, {self.color}"
        return _str

    @staticmethod
    def digit_serializer(text: str) -> int:
        digits = re.sub(r"\D", "", text)
        if not digits:
            return 0
        return int(digits)


def parse_advert(advert: Tag) -> Car:
    description = advert.find("div", class_="ListingItem__description")

    summary_block = description.find("div", class_="ListingItem__summary")
    name = summary_block.find("div", class_="ListingItem__title").text
    link = summary_block.find("a", class_="ListingItemTitle__link").get("href")

    tech_summary = description.find("div", class_="ListingItem__techSummary")
    tech_column_first, tech_column_second = tech_summary.find_all(
        "div", class_="ListingItemTechSummaryDesktop__column"
    )
    
    engine_characteristics_cell, gearbox_cell, car_body_cell = tech_column_first.find_all(
        "div", class_="ListingItemTechSummaryDesktop__cell"
    )
    engine_characteristics = engine_characteristics_cell.text
    gearbox = gearbox_cell.text
    car_body = car_body_cell.text

    second_col_cells = tech_column_second.find_all(
        "div", class_="ListingItemTechSummaryDesktop__cell"
    )
    transmission_type = second_col_cells[0].text
    if len(second_col_cells) == 2:
        color = second_col_cells[1].text
    else:
        color = ""

    price_block = description.find("div", class_="ListingItem__price")
    price_tag = price_block.find("div", class_="ListingItemPrice__content")
    if price_tag is None:
        price_tag = price_block
    price = price_tag.text

    year_block = description.find("div", class_="ListingItem__yearBlock")
    year = year_block.find("div", class_="ListingItem__year").text

    km_age_block = description.find("div", class_="ListingItem__kmAgeAndActionsBlock")
    km_age = km_age_block.find("div", class_="ListingItem__kmAge").text

    car = Car(
        name,
        price,
        km_age,
        year,
        engine_characteristics,
        gearbox,
        car_body,
        transmission_type,
        color,
        link,
    )
    return car

In [47]:
adverts = soup.find_all("div", class_="ListingItem")

len(adverts)

1184

In [63]:
cars_data = [parse_advert(advert) for advert in adverts]

cars_data[:5]

[Mazda CX-9 II Рестайлинг: 6180000₽, 0 км, 2024, 2.5, 231, бензин, автомат, внедорожник 5 дв., полный, ,
 Mazda CX-9 II: 3000000₽, 163112 км, 2019, 2.5, 231, бензин, автомат, внедорожник 5 дв., полный, синий,
 Mazda CX-5 II Рестайлинг: 5550000₽, 0 км, 2023, 2.5, 196, бензин, автомат, внедорожник 5 дв., полный, ,
 Mazda CX-5 II: 2611000₽, 102107 км, 2019, 2.0, 150, бензин, автомат, внедорожник 5 дв., полный, коричневый,
 Mazda CX-5 II: 3127000₽, 72000 км, 2020, 2.0, 150, бензин, автомат, внедорожник 5 дв., полный, чёрный]

In [64]:
duplicates = [item for item, count in Counter(cars_data).items() if count > 1]

print(len(duplicates))
print(duplicates)

0
[]


In [65]:
df = pd.DataFrame([vars(car) for car in cars_data])

df.head()

,name,price,km_age,year,engine_volume,engine_power,fuel_type,gearbox,car_body,transmission_type,color,link
0,Mazda CX-9 II Рестайлинг,6180000,0,2024,2.5,231,бензин,автомат,внедорожник 5 дв.,полный,,https://auto.ru/cars/new/group/mazda/cx_9/2277...
1,Mazda CX-9 II,3000000,163112,2019,2.5,231,бензин,автомат,внедорожник 5 дв.,полный,синий,https://auto.ru/cars/used/sale/mazda/cx_9/1126...
2,Mazda CX-5 II Рестайлинг,5550000,0,2023,2.5,196,бензин,автомат,внедорожник 5 дв.,полный,,https://auto.ru/cars/new/group/mazda/cx_5/2355...
3,Mazda CX-5 II,2611000,102107,2019,2.0,150,бензин,автомат,внедорожник 5 дв.,полный,коричневый,https://auto.ru/cars/used/sale/mazda/cx_5/1125...
4,Mazda CX-5 II,3127000,72000,2020,2.0,150,бензин,автомат,внедорожник 5 дв.,полный,чёрный,https://auto.ru/cars/used/sale/mazda/cx_5/1126...


In [67]:
df.drop("link", axis=1).to_csv(f"{BRAND}_cars_no_link.csv", index=False, encoding="utf-16")